# e-SNLI Heuristic Qualitative Result

In this notebook, we show the qualitative of heuristic attention maps on e-SNLI dataset.

## Setting up

In [1]:
%load_ext autoreload
%autoreload 2

from IPython.display import display, HTML
import sys
import os
from os import path

sys.path.append("./../src")

In [2]:
from modules.logger import init_logging
from modules.logger import log

init_logging(color=True)

In [3]:
!nvidia-smi

Tue Oct 10 15:17:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  On   | 00000000:04:00.0 Off |                  N/A |
| 23%   18C    P8     8W / 250W |      1MiB / 11178MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Parameters

In [4]:
import platform

# Define root folder based on current node (local or server)
node = platform.node()
log.info(f'Current node: {node}')
if node == 'MAC-C02D80HRMD6':
    ROOT = '/Users/dunguyen/Developer/server_backup/historic/2023-06-05'
else:
    ROOT = '/home/dunguyen/RUNS'
    
# model path
LOG_PATH = path.join(ROOT, 'logs')
DATA_CACHE = path.join(ROOT, 'dataset')
MODEL_CACHE = path.join(ROOT, 'models')

10-10-2023 15:17:34 |     INFO   2147724394.py:<cell line: 5>:5  Current node: grele-5.nancy.grid5000.fr


In [5]:
from data_module.esnli_module import ESNLIDM
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

###############
# PREPARE DATA
###############
dm = ESNLIDM(cache_path=DATA_CACHE, batch_size=16, num_workers=16, shuffle=False)
dm.prepare_data()
dm.setup('test')

10-10-2023 15:17:53 |     INFO   esnli_module.py:prepare_data:103  Loaded vocab at /home/dunguyen/RUNS/dataset/esnli/vocab.pt
10-10-2023 15:17:53 |     INFO   esnli_module.py:prepare_data:105  Vocab size: 26578


In [18]:
#####################################
# LOAD HEURISTIC AND ANNOTATION MAPS
#####################################
df = dm.test_set.data
df = df[['id', 'tokens.form.premise', 'tokens.form.hypothesis', 'tokens.norm.premise', 'tokens.norm.hypothesis', 'label', 'heuristic.premise', 'heuristic.hypothesis', 'rationale.premise', 'rationale.hypothesis']].copy()
df.set_index('id', inplace=True)
df.head()

,tokens.form.premise,tokens.form.hypothesis,tokens.norm.premise,tokens.norm.hypothesis,label,heuristic.premise,heuristic.hypothesis,rationale.premise,rationale.hypothesis
id,,,,,,,,,
2677109430.jpg#1r1n,"[This, church, choir, sings, to, the, masses, ...","[The, church, has, cracks, in, the, ceiling, .]","[this, church, choir, sing, to, the, masse, as...","[the, church, have, crack, in, the, ceiling, .]",neutral,"[-1.0000000150474662e+30, 3.064525842666626, 1...","[-1.0000000150474662e+30, 7.628961086273193, -...","[False, False, False, False, False, False, Fal...","[False, False, False, True, True, True, True, ..."
2677109430.jpg#1r1e,"[This, church, choir, sings, to, the, masses, ...","[The, church, is, filled, with, song, .]","[this, church, choir, sing, to, the, masse, as...","[the, church, be, fill, with, song, .]",entailment,"[-1.0000000150474662e+30, 2.79181170463562, 2....","[-1.0000000150474662e+30, 7.628961086273193, -...","[False, False, True, True, True, True, True, F...","[False, False, False, True, True, True, False]"
2677109430.jpg#1r1c,"[This, church, choir, sings, to, the, masses, ...","[A, choir, singing, at, a, baseball, game, .]","[this, church, choir, sing, to, the, masse, as...","[a, choir, singing, at, a, baseball, game, .]",contradiction,"[-1.0000000150474662e+30, 2.5598974227905273, ...","[-1.0000000150474662e+30, 6.388305187225342, 6...","[False, False, False, False, False, False, Tru...","[False, False, True, False, False, True, True,..."
6160193920.jpg#4r1n,"[A, woman, with, a, green, headscarf, ,, blue,...","[The, woman, is, young, .]","[a, woman, with, a, green, headscarf, ,, blue,...","[the, woman, be, young, .]",neutral,"[-1.0000000150474662e+30, 2.597653388977051, -...","[-1.0000000150474662e+30, 5.648240089416504, -...","[False, False, False, False, False, False, Fal...","[False, False, False, True, False]"
6160193920.jpg#4r1e,"[A, woman, with, a, green, headscarf, ,, blue,...","[The, woman, is, very, happy, .]","[a, woman, with, a, green, headscarf, ,, blue,...","[the, woman, be, very, happy, .]",entailment,"[-1.0000000150474662e+30, 2.784580707550049, -...","[-1.0000000150474662e+30, 5.648240089416504, -...","[False, False, False, False, False, False, Fal...","[False, False, False, False, True, False]"


In [19]:
# Normalize heuristic scores
df['heuristic.premise'] = df['heuristic.premise'].apply(lambda x: torch.tensor(x).exp())
df['heuristic.hypothesis'] = df['heuristic.hypothesis'].apply(lambda x: torch.tensor(x).exp())

df['heuristic.premise'] = df['heuristic.premise'].apply(lambda x: x / x.max())
df['heuristic.hypothesis'] = df['heuristic.hypothesis'].apply(lambda x: x / x.max())

In [20]:
import pandas as pd

model_prediction = pd.read_json(LOG_PATH + '/lstm_attention/esnli/run=0_lstm=1/predictions/inference.json')
model_prediction.set_index('id', inplace=True)

# Remove padding mask on attention
model_prediction['a_hat.premise'] = model_prediction.apply(lambda x: [a for a, m in zip(x['a_hat.premise'], x['padding_mask.premise']) if not m], axis=1)
model_prediction['a_hat.hypothesis'] = model_prediction.apply(lambda x: [a for a, m in zip(x['a_hat.hypothesis'], x['padding_mask.hypothesis']) if not m], axis=1)
model_prediction.drop(columns=['padding_mask.premise', 'padding_mask.hypothesis'], inplace=True)

In [21]:
df = df.join(model_prediction[['y_hat', 'a_hat.premise', 'a_hat.hypothesis']])
df = df[df['y_hat'] == df['label']]
df = df[df['label'] != 'neutral']

In [ ]:
# Normalize attention scores
from modules.utils import rescale
df['a_hat.premise'] = df['a_hat.premise'].apply(lambda x: rescale(x).tolist())
df['a_hat.hypothesis'] = df['a_hat.hypothesis'].apply(lambda x: rescale(x).tolist())

In [29]:
df['a_true'] = df.apply(lambda x: [0.] + x['rationale.premise'] + [0.] + x['rationale.hypothesis'], axis=1)
df['a_hat'] = df.apply(lambda x: [0.] + x['a_hat.premise'] + [0.] + x['a_hat.hypothesis'], axis=1)
df['a_heu'] = df.apply(lambda x: [0.] + x['heuristic.premise'].tolist() + [0.] + x['heuristic.hypothesis'].tolist(), axis=1)
df['tokens.form'] = df.apply(lambda x: ['<b>Premise: </b>'] + x['tokens.form.premise'] + ['<br/><b>Hypothesis: </b>'] + x['tokens.form.hypothesis'], axis=1)
df['tokens.norm'] = df.apply(lambda x: ['<b>Premise: </b>'] + x['tokens.norm.premise'] + ['<br/><b>Hypothesis: </b>'] + x['tokens.norm.hypothesis'], axis=1)

In [ ]:
from tqdm.notebook import tqdm
from modules.utils import highlight
import shutil

# Remove the previous existing folder
PROJECT = 'qualitative_heuristic_esnli'
html_dir = path.join(ROOT, PROJECT)
log.info(f'Save qualitative results at {html_dir}')
if os.path.exists(html_dir) and os.path.isdir(html_dir):
    log.info(f'Removing existing folder {html_dir}')
    shutil.rmtree(html_dir)

# Generate each comparison into a file:
for idx, row in tqdm(df.iterrows(), total=len(df)):
    
    # ignore if label is 0
    if row['label'] == 'neutral': continue
    
    html = """
    <html>
    <head>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@4.6.2/dist/css/bootstrap.min.css" integrity="sha384-xOolHFLEh07PJGoPkLv1IbcEPTNtaed2xpHsD9ESMhqIYd0nLMwNLD69Npy4HI+N" crossorigin="anonymous">
    <style>
    table, th, td {
      border:solid black;
      border-collapse: collapse;
      padding: 0px 5px 0px 5px;
    }
    </style></head>
    <body>
    """
    html += '<table style="font-size:120%;" cellspacing=0>'
    html += f'<caption>Dataset: e-SNLI - Instance ID: {idx}</caption>'
    html += f'<tr><th style="width:100px;">Explainer</th> <th style="width:500px;">Explanation</th> <th style="width:100px;">Label</th></tr>'
    
    # Annotation map
    map_viz = highlight(row['tokens.form'], row['a_true'], normalize_weight=False)
    html += f'<tr><td style="text-align:right;"> Annotation Map</td><td>{map_viz}</td><td rowspan="3" style="text-align:center"> {row["label"]} </td></tr>'
    
    # Attention map
    map_viz = highlight(row['tokens.norm'], row['a_hat'], normalize_weight=False)
    html += f'<tr><td style="text-align:right;"> Attention Map</td><td>{map_viz}</td></tr>'
    
    # Heuristic map
    map_viz = highlight(row['tokens.form'], row['a_heu'], normalize_weight=True)
    html += f'<tr><td style="text-align:right;"> Heuristic Map</td><td>{map_viz}</td></tr>'
    
    # End
    html += '</table>'
    html += '</body></html>'

    fpath_html = path.join(html_dir, f'{idx}.html')
    os.makedirs(html_dir, exist_ok=True)
    with open(fpath_html, 'w') as f:
        f.write(html)

ModuleNotFoundError: No module named 'modules'